In [1]:
import numpy as np

In [2]:
import math

In [6]:
a = np.ones((3,3))
a[0,1] = 2
a[0,2] = 4
a[1,0] = 3
a[2,0] = 5
a[1,1] = 10
a[1,2] = 10
a[2,1] = 5
a[2,2] = 2
a

array([[ 1.,  2.,  4.],
       [ 3., 10., 10.],
       [ 5.,  5.,  2.]])

In [7]:
nx,ny = a.shape
int_img = np.ones((nx,ny))
int_img[0,0] = a[0,0]
for i in range(nx):
    for j in range(ny):
        if j > 0 and i > 0:
            int_img[i,j] = int_img[i-1,j] + sum([ a[i,k] for k in range(j+1)])
            print(i,j)
            print(int_img[i-1,j])
            print(sum([ a[i,k] for k in range(j+1)]))
        elif j==0 and i>0:
            int_img[i,0] = int_img[i-1,0] + a[i,0]
        elif i==0 and j>0:
            int_img[0,j] = int_img[0,j-1] + a[0,j]

1 1
3.0
13.0
1 2
7.0
23.0
2 1
16.0
10.0
2 2
30.0
12.0


In [8]:
int_img

array([[ 1.,  3.,  7.],
       [ 4., 16., 30.],
       [ 9., 26., 42.]])

In [16]:
def feature_list(nx=64,ny=64,height=2,width=1,stride=1):
    f12_list = []
    f21_list = []
    for j in range(0,ny,stride):
        for i in range(0,nx, stride):
            ### 1 by 2 (x by y)
            if i+stride+width <= nx-1 and j+height <= ny-1:
                #shaded
                xs, ys  = i,j # upper left
                _xs,_ys = i+width, j+height# bottom right
                # unshaded
                xu, yu  = i+stride,j # upper left
                _xu,_yu = i+stride+width, j+height# bottom right
                # shaded + unshaded
                f12_list += [[[xs, ys],[_xs, _ys],[xu, yu],[_xu, _yu]]]
            ### 2 by 1 (x by y)
            if i+stride+width <= nx-1 and j+stride+width <= ny-1:
                #shaded
                xs, ys  = i,j # upper left
                _xs,_ys = i+height, j+width# bottom right
                # unshaded
                xu, yu  = i,j+stride # upper left
                _xu,_yu = i+height, j+stride+width# bottom right
                # shaded + unshaded
                f21_list += [[[xs, ys],[_xs, _ys],[xu, yu],[_xu, _yu]]]
    return f12_list+f21_list

In [10]:
flist = feature_list(nx=3, ny=3, height=1)
len(flist)

3

In [11]:
flist

[[[0, 0], [1, 1], [1, 0], [2, 1]],
 [[0, 1], [1, 2], [1, 1], [2, 2]],
 [[0, 0], [1, 1], [0, 1], [1, 2]]]

In [17]:
def compute_feature(int_img_rep, feat_lst, feat_idx):
    """ int_img_rep: the N x 64 x 64 numpy matrix of the integral image representation
        feat_lst: list of features
        feat_idx: integer, index of a feature (in feat_lst)
        Returns: an N x 1 numpy matrix of the feature evaluations for each image
    """
    n, nx, ny = int_img_rep.shape
    feat_eval = []

    def _compute_intensity(int_img, ul, rb):
        """
        compute intensity
        x: upper left [ix, jx]
        y: bottom right [iy, jy]
        
        *(ix,iy)---*
        |          |
        |          |
        *----------*(jx,jy)
        """
        ix, iy = ul # upper left x,y
        jx, jy = rb #  bottom right x, y
        if ix-1 >= 0 and iy-1 >= 0  :
            intensity = int_img[jx,jy] + int_img[ix-1,iy-1] - int_img[ix-1,jy] - int_img[jx,iy-1]
        elif ix-1 >=0 and iy-1<0:
            intensity = int_img[jx,jy] - int_img[ix-1,jy]
        elif ix-1 < 0 and iy-1>=0:
            intensity = int_img[jx,jy]  - int_img[jx,iy-1]
        elif ix-1 < 0 and iy-1< 0:
            intensity = int_img[jx,jy] 
        return intensity
    
    for img in int_img_rep:
        """
        x: upper left in shaded
        y: bottom right in shaded
        z: upper left in unshaded
        s: bottom right in unshaded
        """
        x,y,z,s = feat_lst[feat_idx] 
        shaded   = _compute_intensity(img, x,y) 
        unshaded = _compute_intensity(img, z,s)
        feat_eval +=[shaded - unshaded]
    
    return np.asarray(feat_eval)

## Question 5

### 5.0 Adaboost

In [ ]:
def ada_boost(x):
    ndata = 10
    nt = 10
    D = np.zeros((ndata,nt))
    for i in len(ndata):
        D[i,0] = 1/ndata
        for it in range(nt):
            ht = 0 #epsilon ??
            alpha_t = 0.5*math.log((1-epsilon)/epsilon)
            zt = 2*np.sqrt((epsilon*(1-epsilon)))
            
            for j in in len(ndata):
                D[j,it+1] = D[j, it]* math.exp(-1*alpha_t*y_t*ht(ix))/zt
            # f
            f += alpha_t*ht  
    return h = sgn(f)

In [ ]:
def ada_boost(x):
    """
    D: weights
    epsilon:Classifier’s error rate.
            This is just the number of misclassifications over the training set 
            divided by the training set size.
    """
    ndata = 10
    nt = 10
    D = np.zeros((ndata,nt))
    D[:,0] = 1/len(ndata)
    for i in len(ndata):
        
        for it in range(nt):
            ht = 0 #epsilon ??
            alpha_t = 0.5*math.log((1-epsilon)/epsilon)
            zt = 2*np.sqrt((epsilon*(1-epsilon)))
            
            for j in in len(ndata):
                D[j,it+1] = D[j, it]* math.exp(-1*alpha_t*y_t*ht(ix))/zt
            # f
            f += alpha_t*ht  
    return h = sgn(f)

### 5.1

In [20]:
def opt_p_theta(int_img_rep, feat_lst, weights, feat_idx):
    """
    p = {+1,-1}
    theta = threshold
    """
    p, theta = 0,0
    #value of feature_idx on example 1-All
    eval_list = [] # f(x_sigma)
    for i in range(len(feat_lst)):
        eval_list += [compute_feature(int_img_rep=int_img_rep, feat_lst=feat_lst, feat_idx=i)]
    #permutation
    eval_list.sort() # f(x_sigma1) < f(x_sigma2) ... < f(x_sigmaN)
    
    #potential theta
    tmp_theta = []
    theta_left = eval_list[feat_idx]
    if feat_idx+ 1 <= len(eval_list):
        theta_right = eval_list[feat_idx+1]
    else:
        theta_right = theta_left
    
    # decide theta
    """
     f(x_sigma_j) <= theta <= f(x_sigma_j+1) j:= feat_idx
    """
    theta = theta_left + (theta_right-theta_left)/2
    
    #decide p
    p=-1
    
    return p, theta

In [13]:
# first wegiht
weight = np.ones((3))*(1/3)
weight

array([0.33333333, 0.33333333, 0.33333333])

In [22]:
opt_p_theta(int_img.reshape(1,3,3), flist, weight, 1)

(-1, array([-4.]))

#### Practice for question 5

In [ ]:
# Assume now is time step=1
# weight is equal
# number of data is 10
# matrix size is 24 by 24

In [3]:
nfile = 10
npatch=24
imgs = np.ones((nfile,npatch,npatch))

In [4]:
def compute_integral_image(imgs):
    """
    IN: imgs. np.ndarray, shape [#images, 64,64]
    """
    
    def _compute_int_img_unit(img):
        """
            compute integral image for one image 
        """
        nx, ny = img.shape
        int_img = np.ones((nx,ny))
        int_img[0,0] = img[0,0]
        for i in range(nx):
            for j in range(ny):
                if j > 0 and i > 0:
                    int_img[i,j] = int_img[i-1,j] + sum([ img[i,k] for k in range(j+1)])
                elif j==0 and i>0:
                    int_img[i,0] = int_img[i-1,0] + img[i,0]
                elif i==0 and j>0:
                    int_img[0,j] = int_img[0,j-1] + img[0,j]
        return int_img
    
    nimg, nx, ny = imgs.shape
    int_img_list = []
    for img in imgs:
        int_img_list += [_compute_int_img_unit(img).reshape(1,nx,ny)]
    int_imgs = np.concatenate(int_img_list, axis=0)
    return int_imgs

In [5]:
int_imgs = compute_integral_image(imgs)

In [7]:
int_imgs.shape

(10, 24, 24)

In [8]:
w = np.ones((nfile))*(1/nfile) # weight

In [9]:
w

array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1])

In [15]:
### make sudo label
y_true = np.ones((nfile))
for i in range(len(y_true)):
    if i < int(nfile/4):
        y_true[i] = -1
    else:
        y_true[i] = 1
np.random.shuffle(y_true)
y_true

array([-1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1.])

In [19]:
#### feature
flist = feature_list(nx=24,ny=24)
len(flist)

968

In [23]:
def dev_opt_p_theta(int_img_rep, feat_lst, weights, feat_idx, y_true):
    """
    p = {+1,-1}
    theta = threshold
    """
    p, theta = 0,0
    #value of feature_idx on example 1-All
    eval_list = [] # f(x_sigma)
    for i in range(len(feat_lst)):
        eval_list += [compute_feature(int_img_rep=int_img_rep, feat_lst=feat_lst, feat_idx=i)]
    #permutation
    eval_list.sort() # f(x_sigma1) < f(x_sigma2) ... < f(x_sigmaN)
    
    #potential theta
    tmp_theta = []
    theta_left = eval_list[feat_idx]
    if feat_idx+ 1 <= len(eval_list):
        theta_right = eval_list[feat_idx+1]
    else:
        theta_right = theta_left
    
    # decide theta
    """
     f(x_sigma_j) <= theta <= f(x_sigma_j+1) j:= feat_idx
    """
    theta = theta_left + (theta_right-theta_left)/2
    
    #decide p
    plus_vec  = np.ones(len(y_true))## length is y_true
    minus_vec = np.ones(len(y_true))
    plus_vec[np.where(y_true == 1)] = 1 # binary vector
    minus_vec[np.where(y_true == -1)] = 1
    
    s_plus = np.sum(np.dot(weights, plus_vec)[:feat_idx])
    s_minus = np.sum(np.dot(weights, minus_vec)[:feat_idx])
    t_plus = np.sum(np.dot(weights, plus_vec))
    t_minus = np.sum(np.dot(weights, minus_vec))
    
    epsilon = min((s_plus+(t_minus-s_minus)), (s_minus+(t_plus-s_plus)))
    p = np.sign(epsilon)
    
    return p, theta

In [21]:
a = np.asarray([[1,2],[3,4]])
b = np.asarray([1,2])
print(a,b)
np.dot(a,b)

[[1 2]
 [3 4]] [1 2]


array([ 5, 11])

In [22]:
c = np.dot(a,b)
np.sum(c)

16

In [26]:
dev_opt_p_theta(int_imgs, flist, w, 10, y_true)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()